In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import cace
from cace import data

In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
torch.set_default_dtype(torch.float64)

In [ ]:
from ase import Atoms

In [ ]:
atom=Atoms(numbers=np.array([8, 1, 1, 1]),
    positions=np.array(
        [
            [0.0452, -2.02, 0.0452],
            [1.0145, 0.034, 0.0232],
            [0.0111, 1.041, -0.010],
             [-0.0111, -0.041, 0.510],
        ]
    ),
     pbc=False)

# Created the rotated environment
rot = R.from_euler("z", 70, degrees=True).as_matrix()
positions_rotated = np.array(rot @ atom.positions.T).T

rot = R.from_euler("x", 10.6, degrees=True).as_matrix()
positions_rotated = np.array(rot @ positions_rotated.T).T

rot = R.from_euler("y", 190, degrees=True).as_matrix()
positions_rotated = np.array(rot @ positions_rotated.T).T

atom_rotated=Atoms(numbers=np.array([8, 1, 1, 1]),
 positions=positions_rotated,
     pbc=False)

In [ ]:
cutoff = 5.0

In [ ]:
atomic_data = data.AtomicData.from_atoms(atom, cutoff=cutoff)
atomic_data2 = data.AtomicData.from_atoms(atom_rotated, cutoff=cutoff)

In [ ]:
atomic_data.positions

In [ ]:
atomic_data2.positions

In [ ]:
from cace.representations.cace_representation import Cace

In [ ]:
atomic_data

In [ ]:
from cace.modules import CosineCutoff, MollifierCutoff, PolynomialCutoff
from cace.modules import BesselRBF, GaussianRBF, GaussianRBFCentered, ExponentialDecayRBF

In [ ]:
radial_basis = BesselRBF(cutoff=cutoff, n_rbf=5, trainable=False)
#radial_basis = ExponentialDecayRBF(n_rbf=4, cutoff=cutoff, prefactor=1, trainable=True)
cutoff_fn = CosineCutoff(cutoff=cutoff)

In [ ]:
cace_representation = Cace(
    zs=[1,8],
    n_atom_basis=2,
    cutoff=cutoff,
    cutoff_fn=cutoff_fn,
    radial_basis=radial_basis,
    max_l=6,
    max_nu=4,
    num_message_passing=2,
    timeit=True
           )

In [ ]:
cace_result = cace_representation(atomic_data)

In [ ]:
cace_result2 = cace_representation(atomic_data2)

In [ ]:
features = cace_result['node_feats']

In [ ]:
features2 = cace_result2['node_feats']

In [ ]:
torch.allclose(features, features2, rtol=1e-05, atol=1e-05)

In [ ]:
features.shape

In [ ]:
features[0,2,:,0,0]

In [ ]:
features[0,1,:,0,1]

In [ ]:
features[0,1,:,0,2]

In [ ]:
features2[0,1,:,0,1]